<a href="https://colab.research.google.com/github/2303a51355/High-performance-computing/blob/main/2303a51355.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BARRIER**


barrier makes all threads wait until every thread reaches the same point.

No thread can go ahead until all arrive

In [ ]:
%%writefile barrier.c
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel
    {
        int id = omp_get_thread_num();

        printf("Thread %d before barrier\n", id);

        #pragma omp barrier

        printf("Thread %d after barrier\n", id);
    }
    return 0;
}


Overwriting barrier.c


In [ ]:
!gcc barrier.c -fopenmp -o barrier


In [ ]:
!./barrier


Thread 1 before barrier
Thread 0 before barrier
Thread 1 after barrier
Thread 0 after barrier


**CRITICAL**

Allows only ONE thread at a time to execute a block.

Used to protect shared data from race condition.

In [ ]:
%%writefile critical.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;

    #pragma omp parallel
    {
        #pragma omp critical
        {
            total++;
            printf("Thread %d -> total = %d\n",
                   omp_get_thread_num(), total);
        }
    }

    printf("Final total = %d\n", total);
    return 0;
}


Overwriting critical.c


In [ ]:
!gcc critical.c -fopenmp -o critical


In [ ]:
!./critical


Thread 0 -> total = 1
Thread 1 -> total = 2
Final total = 2


**ATOMIC**

Similar to critical but only for single simple statement

Faster than critical

In [ ]:
%%writefile atomic.c
#include <stdio.h>
#include <omp.h>

int main() {
    int sum = 0;

    #pragma omp parallel
    {
        #pragma omp atomic
        sum += 1;
    }

    printf("Sum = %d\n", sum);
    return 0;
}


Overwriting atomic.c


In [ ]:
!gcc atomic.c -fopenmp -o atomic


In [ ]:
!./atomic


Sum = 2


**ORDERED**

Used inside parallel loops

Ensures some part executes in sequential order

In [ ]:
%%writefile ordered.c
#include <stdio.h>
#include <omp.h>

int main() {

    #pragma omp parallel for ordered
    for(int i = 0; i < 5; i++) {

        #pragma omp ordered
        {
            printf("Ordered Output: %d by thread %d\n",
                   i, omp_get_thread_num());
        }
    }

    return 0;
}


Overwriting ordered.c


In [ ]:
!gcc ordered.c -fopenmp -o ordered


In [ ]:
!./ordered


Ordered Output: 0 by thread 0
Ordered Output: 1 by thread 0
Ordered Output: 2 by thread 0
Ordered Output: 3 by thread 1
Ordered Output: 4 by thread 1


openmp(lock)


In [ ]:
%%writefile lock_example.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;
    omp_lock_t lock;

    // Initialize the lock
    omp_init_lock(&lock);

    #pragma omp parallel num_threads(4)
    {
        for(int i = 0; i < 5; i++) {
            // Acquire lock before updating total
            omp_set_lock(&lock);
            total++;
            printf("Thread %d updated total = %d\n", omp_get_thread_num(), total);
            omp_unset_lock(&lock);  // Release lock
        }
    }

    // Destroy the lock
    omp_destroy_lock(&lock);

    printf("Final total = %d\n", total);
    return 0;
}


Writing lock_example.c


In [ ]:
!gcc lock_example.c -fopenmp -o lock_example


In [ ]:
!./lock_example


Thread 3 updated total = 1
Thread 3 updated total = 2
Thread 3 updated total = 3
Thread 3 updated total = 4
Thread 3 updated total = 5
Thread 1 updated total = 6
Thread 1 updated total = 7
Thread 1 updated total = 8
Thread 1 updated total = 9
Thread 1 updated total = 10
Thread 2 updated total = 11
Thread 2 updated total = 12
Thread 2 updated total = 13
Thread 2 updated total = 14
Thread 2 updated total = 15
Thread 0 updated total = 16
Thread 0 updated total = 17
Thread 0 updated total = 18
Thread 0 updated total = 19
Thread 0 updated total = 20
Final total = 20


openmp(nolock)

In [ ]:
%%writefile nolock_example.c
#include <stdio.h>
#include <omp.h>

int main() {
    int total = 0;

    #pragma omp parallel num_threads(4)
    {
        for(int i = 0; i < 5; i++) {
            // No lock here
            total++;
            printf("Thread %d updated total = %d\n", omp_get_thread_num(), total);
        }
    }

    printf("Final total = %d\n", total);
    return 0;
}


Writing nolock_example.c


In [ ]:
!gcc nolock_example.c -fopenmp -o nolock_example


In [ ]:
!./nolock_example


Thread 2 updated total = 1
Thread 2 updated total = 3
Thread 2 updated total = 4
Thread 3 updated total = 2
Thread 3 updated total = 6
Thread 3 updated total = 7
Thread 3 updated total = 8
Thread 3 updated total = 9
Thread 1 updated total = 10
Thread 1 updated total = 11
Thread 1 updated total = 12
Thread 1 updated total = 13
Thread 1 updated total = 14
Thread 2 updated total = 5
Thread 2 updated total = 15
Thread 0 updated total = 16
Thread 0 updated total = 17
Thread 0 updated total = 18
Thread 0 updated total = 19
Thread 0 updated total = 20
Final total = 20
